In [16]:
#este es el primer paso, a partir de la lista de palabras en inglés se consiguen las traducciones usando
#Google Cloud API Translate
#el input es un archivo csv de una columna y el output es un archivo csv de tres columnas:
#index, english and translation from english to another language
import os
import csv
import pandas as pd
from pathlib import Path

def translate_text(target, text):
    """Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    # print(u"Text: {}".format(result["input"]))
    # print(u"Translation: {}".format(result["translatedText"]))
    # print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
    return result["translatedText"]


def main(input_file, translate_to, code):
    """
    Translate a text file and save as a CSV file
    using Google Cloud Translation API
    """
    input_file_path = Path(input_file)
    target_lang = translate_to
    new_directory="/home/alonso/PalabrasTraducidasGoogleAPI/"
    #output_file_path = input_file_path.with_name(f'{target_lang}{"-"}{input_file_path.stem}{input_file_path.suffix}')
    output_file_path = new_directory+code+"_"+target_lang+"_"+os.path.basename(input_file_path)
    
    with open(input_file_path) as f:
        list_lines = f.readlines()
        total_lines = len(list_lines)
    with open(output_file_path, 'w') as csvfile:
        my_writer = csv.writer(csvfile, delimiter=',', quotechar='"')
        my_writer.writerow(['id', 'original_text', 'translated_text'])

        for i, each_line in enumerate(list_lines):
            line_id = f'{i + 1:04}'
            original_text = each_line.strip('\n')  # Strip for the writer(*).
            translated_text = translate_text(
                target=target_lang,
                text=each_line)
            my_writer.writerow([line_id, original_text, translated_text])  # (*)
            # Progress monitor, non-essential.

          
# The path to the directory containing the files
directory = '/media/alonso/3361-6630/neologisms/prueba'
            
df = pd.read_csv('ISO639GoogleEpitran.csv', sep='\t')
if __name__ == '__main__':
    for index, row in df.iterrows():
            output_lang = row['639-1']
            ISO15924script = row['Code']
            for filename in os.listdir(directory):
                origin_file = os.path.join(directory, filename)
                main(input_file=origin_file, translate_to=output_lang, code=ISO15924script)

In [1]:
#Este es el segundo paso. Este script recibe como input un archivo csv con tres columnas: numeración, english words y
#the same words translated to another language. The output es un archivo tsv con cuatro columnas, las tres anteriores
#más la transcripción IPA de la palabra en el idioma traducido
import os
import csv
import pandas as pd
from csv import writer
from csv import reader
from epitran.backoff import Backoff
#directory="/home/alonso/noprocesados/"
directory="/home/alonso/PalabrasTraducidasGoogleAPI2/" #directorio donde están las palabras traducidas
new_directory="/home/alonso/PalabrasTraducidasTranscritasIPA2/"
df = pd.read_csv('ISO639google.csv', sep='\t')
for filename in sorted(os.listdir(directory)):
    # Open the input_file in read mode and output_file in write mode
    language = filename[9:11] #OJO en la versión definitiva sería [9:11]
    code = filename[:8]        
    backoff = Backoff([code])
    newfile = new_directory+"IPA_"+filename 
    with open(directory+filename, 'r') as f, \
        open(newfile, 'w', newline='') as w:
        # Create a csv.reader object from the input file object
        csv_reader = reader(f)
        # Create a csv.writer object from the output file object
        csv_writer = writer(w, delimiter='\t', lineterminator='\n')
        # Read each row of the input csv file as list
    #type(csv_reader[0])
        for row in csv_reader:
            IPA=backoff.trans_list(row[2])
        #este procedimiento coloca row e IPA en la misma fila 
            row.append(IPA)
        # Add the updated row / list to the output file oxfioxfi
            csv_writer.writerow(row)   

In [2]:
#este es el tercer paso
#el output de epitran no es una lista sino una cadena, por ello era necesario tokenisar para evitar símbolos extraños
#simplemente aplicando tokenise fue suficiente para resolver el problema
#el input de este programa es un directorio con archivos .tsv de cuatro columnas y el output es un archivo .tsv
#de seis columnas: contador por file,english,filename,idioma,word,IPA 
#SE RESOLVIO EL index ERROR añadiendo una condición en:
#if tokenise(pron)[-1] not in NoAlFinalPalabra:
#se resolvió el VALUE ERROR: not enough values to unpack (expected 4, got 1) verificando que len(row)=4
from ipatok import tokenise
import os
import csv
#se retiró β, se reemplazaron tʃʼ,tʃʰ,tʃ por "č","čʰ","č'" 
# se añadio "/" dado que ese caracter acompaña al formato IPA en los archivos de la carpeta csv
IPAquechua=["æ","ɑ","a","ɪ","i","e","ɛ","u","U","o","ʊ","ɔ","h","j","k","k̚","ɣ","x","kʰ","kx","k'","l","ʎ","m","n","ɲ","p","ɸ","pʰ","pɸ","p'","q","ɢ","X","qʰ","q'","ɾ","s","ʃ","t","tʰ","ts","θ","t'","č","čʰ","č'","w","W"]
NoAlFinalPalabra=["h","ñ","č'","čʰ","kʰ","k'","pʰ","p'","qʰ","q'","t","tʰ","t'"] 
alofonos=["ɛ","e","o","ɔ"]
uvular=["q","q'","qʰ"]
vowels=["æ","ɑ","a","ɪ","i","e","ɛ","u","U","o","ʊ","ɔ"]
consonants=["h","j","k","k̚","ɣ","x","kʰ","kx","k'","l","ʎ","m","n","ɲ","p","β","ɸ","pʰ","pɸ","p'","q","ɢ","X","qʰ","q'","ɾ","s","ʃ","t","tʰ","ts","θ","t'","č","čʰ","č'","w"]
directory="/home/alonso/PalabrasTraducidasTranscritasIPA2/"
#directory='/media/alonso/3361-6630/neologisms/GoogleAPIcloudEpitran/'

for filename in sorted(os.listdir(directory)):
    with open(directory+filename,'r') as csvfile:
        reader = csv.reader(csvfile,delimiter='\t')
        count=0
        for row in reader:
            if len(row) == 4:
                index, english, word, pron = row
                #la condición debajo se coloca para evitar ValueError: The string starts with a tie bar: '͡' 
                if "'͡" not in pron:
                    #verificar que la palabra examinada solamente contenga fonemas quechuas
                    if all([char in IPAquechua for char in tokenise(pron)]): 
                        #verificar que no haya consonantes prohibidas al final de palabra
                        if len(tokenise(pron)) > 0 and tokenise(pron)[-1] not in NoAlFinalPalabra:
                        #if tokenise(pron)[-1] not in NoAlFinalPalabra:
                            with open("filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologisms2.tsv", "a") as f: 
                            #si la palabra NO tiene sonidos alofonos, no hace falta verificas si tiene uvulares
                                if all([char not in alofonos for char in tokenise(pron)]):
                                    for i in range(len(tokenise(pron)) - 1): # i is the current index
                                        #verificar si la palabra tiene diptongos
                                        if tokenise(pron)[i + 1] in vowels and tokenise(pron)[i] in vowels:
                                            break
                                        else:
                                            if i == len(tokenise(pron))-2:
                                                for i in range(len(tokenise(pron)) - 1): # i is the current index
                                                    #verificar si la palabra tiene consonantes consecutivas
                                                    if tokenise(pron)[i + 1] in consonants and tokenise(pron)[i] in consonants:
                                                        break
                                                    else:
                                                        if i == len(tokenise(pron))-2:
                                                            count+=1
                                                            f.write(f"{count}\t{english}\t{filename}\t{filename[13:15]}\t{word}\t{tokenise(pron)}\n")
                                                            #print(count,word,pron)
                                else:
                                    #la palabra tiene alofonos, verificar si tiene uvulares 
                                    if any([char in uvular for char in tokenise(pron)]):
                                        #la palabra tiene alofonos y uvulares, verificar si tiene diptongos
                                        for i in range(len(tokenise(pron)) - 1): # i is the current index
                                            if tokenise(pron)[i + 1] in vowels and tokenise(pron)[i] in vowels:
                                                break
                                            else:
                                                if i == len(tokenise(pron))-2:
                                                    for i in range(len(tokenise(pron)) - 1): # i is the current index
                                                        #verificar si la palabra tiene consonantes consecutivas
                                                        if tokenise(pron)[i + 1] in consonants and tokenise(pron)[i] in consonants:
                                                            break
                                                        else:  
                                                            if i == len(tokenise(pron))-2:
                                                                count+=1
                                                                f.write(f"{count}\t{english}\t{filename}\t{filename[13:15]}\t{word}\t{tokenise(pron)}\n")


In [21]:
#este es el cuarto paso
#convertir la transcripción IPA en una palabra quechua
#se añade una columna al archivo tsv input, dicha columna contiene la palabra quechua correspondiente a la transcripción IPA
import os
import pandas as pd
DiccionarioFonemaGrafemaQuechua={"æ":"a","ɑ":"a","a":"a","ɪ":"i","i":"i","e":"i","ɛ":"i","u":"u","U":"u","o":"u","ʊ":"u","ɔ":"u","h":"h","j":"y","k":"k","k̚":"k","ɣ":"k","x":"k","kʰ":"kh","kx":"kh","k'":"k'","l":"l","ʎ":"ll","m":"m","n":"n","ɲ":"ñ","p":"p","ɸ":"p","pʰ":"ph","pɸ":"ph","p'":"p'","q":"q","ɢ":"q","X":"q","qʰ":"qh","q'":"q'","ɾ":"r","s":"s","ʃ":"sh","t":"t","tʰ":"th","ts":"th","θ":"th","t'":"t'","č":"ch","čʰ":"chh","č'":"ch'","w":"w","W":"w"}
#df=pd.read_csv("filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologisms.tsv", sep='\t')
df=pd.read_csv("/media/alonso/3361-6630/neologisms/outputs_2nd_paper/filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologismsJoint.tsv", sep='\t', names=["ordinal", "english", "filename", "language", "word", "IPA"])
# la siguiente linea creó una columna, obviamente se ejecuta una sola vez
df.insert(loc=6,column="qichwa",value="")
palabra_escrita_en_quechua=""
for index,word in enumerate(df["IPA"].values):
    word = str(word)
    for letter in tokenise(word):
        if letter != " ":
        #buscamos en el diccionario el valor equivalente al caracter IPA
            palabra_escrita_en_quechua+=DiccionarioFonemaGrafemaQuechua[letter]
    #despues de construida la palabra quechua, la añadimos al dataframe, en la misma fila
    df.at[index,"qichwa"]=palabra_escrita_en_quechua
    palabra_escrita_en_quechua=""
 #grabar en el archivo GlosaTSV
df.to_csv("filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologismsQichwaWords.tsv", sep='\t')

In [5]:
#este es el quinto paso, el reporte de palabras encontradas por idioma
import os
import pandas as pd
df2=pd.read_csv("filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", sep='\t')
directory="/home/alonso/PalabrasTraducidasTranscritasIPA/"
#df['language'].value_counts().to_frame()
#df['language'].value_counts().to_frame().to_csv('language_counts.csv', index=False)
#df['language'].value_counts().to_frame().reset_index().columns = ['language', 'count']
#df.to_csv('language_counts.csv', index=False)
df2['language'].value_counts().to_frame().reset_index().to_csv('ReportQichwaWords2.csv', index=False)
#añadir cantidad total de palabras examinadas por idioma
df=pd.read_csv("ReportQichwaWords2.csv")
df.insert(loc=2,column="words",value="")
tags = {}

for file in os.listdir(directory):
    if file.endswith(".csv"):
        tag = file[13:15]
        if tag not in tags:
            tags[tag] = 0
        with open(os.path.join(directory, file), "r") as f:
            data = f.read().splitlines()
            tags[tag] += len(data)

for tag, row_count in tags.items():
    for index,word in enumerate(df["index"].values):
        if word == tag:
            df.at[index,"words"]=row_count
    #print(f"{tag}: {row_count}")
df['percentage'] = df['language'] / df['words'] * 100
df = df.round(2)
df.to_csv("ReportQichwaWords2.csv", sep='\t',index=False)    

In [15]:
#este es el quinto paso, el reporte de palabras encontradas por idioma
#este script funciona para los archivos en tsvbroad
import os
import pandas as pd
df2=pd.read_csv("file2tsvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", sep='\t')
directory="/media/alonso/3361-6630/neologisms/tsvbroad/"
#df['language'].value_counts().to_frame()
#df['language'].value_counts().to_frame().to_csv('language_counts.csv', index=False)
#df['language'].value_counts().to_frame().reset_index().columns = ['language', 'count']
#df.to_csv('language_counts.csv', index=False)
df2['fil'].value_counts().to_frame().reset_index().to_csv('ReportQichwaWordsTSV.csv', index=False)
#añadir cantidad total de palabras examinadas por idioma
df=pd.read_csv("ReportQichwaWordsTSV.csv")
df.insert(loc=2,column="words",value="")

for file in sorted(os.listdir(directory)):
        for index,fil in enumerate(df["index"].values):
            with open(os.path.join(directory, file), "r") as f:
                data = f.read().splitlines()
            #tags[tag] += len(data)
                if fil == file[:3]:
                    df.at[index,"words"]=len(data)
    #print(f"{tag}: {row_count}")
df['percentage'] = df['fil'] / df['words'] * 100
df = df.round(2)
df.to_csv("ReportQichwaWordsTSV.csv", sep='\t',index=False)    

In [16]:
#este es el quinto paso, el reporte de palabras encontradas por idioma
#este script funciona para los archivos en csv
import os
import pandas as pd
df2=pd.read_csv("file1csvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", sep='\t')
directory="/media/alonso/3361-6630/neologisms/csv/"
#df['language'].value_counts().to_frame()
#df['language'].value_counts().to_frame().to_csv('language_counts.csv', index=False)
#df['language'].value_counts().to_frame().reset_index().columns = ['language', 'count']
#df.to_csv('language_counts.csv', index=False)
df2['filename'].value_counts().to_frame().reset_index().to_csv('ReportQichwaWordsCSV.csv', index=False)
#añadir cantidad total de palabras examinadas por idioma
df=pd.read_csv("ReportQichwaWordsCSV.csv")
df.insert(loc=2,column="words",value="")

for file in sorted(os.listdir(directory)):
        for index,filename in enumerate(df["index"].values):
            with open(os.path.join(directory, file), "r") as f:
                data = f.read().splitlines()
            #tags[tag] += len(data)
                if filename == file:
                    df.at[index,"words"]=len(data)
    #print(f"{tag}: {row_count}")
df['percentage'] = df['filename'] / df['words'] * 100
df = df.round(2)
df.to_csv("ReportQichwaWordsCSV.csv", sep='\t',index=False)    

In [6]:
#este script me permitió seleccionar los idiomas a los cuales todavía no había traducido las palabras en inglés
#el archivo seleccionado se subió a la instancia Google para correr el primer paso
#el script correctamente encontró 48 idiomas faltantes
import csv

def delete_rows(csv_file, value):
    with open(csv_file, "r") as f:
        reader = csv.reader(f, delimiter='\t', lineterminator='\n')
        data = list(reader)

    for row in data:
        if row[1] == value:
            del data[data.index(row)]

    with open(csv_file, "w") as f:
        writer = csv.writer(f, delimiter='\t', lineterminator='\n')
        writer.writerows(data)

if __name__ == "__main__":
    csv_file = '/media/alonso/3361-6630/ISO639google.csv'
    with open('/media/alonso/3361-6630/neologisms/ReportQichwaWords.csv', 'r') as f:
            csv_reader = reader(f, delimiter='\t', lineterminator='\n')
            for row in csv_reader:
                value = row[0]
                delete_rows(csv_file, value)

In [7]:
#este script me permitió descubrir porque el script principal no encontraba palabras candidatas
#descubrí que el output de epitran no es una lista sino una cadena, por ello era necesario tokenisar para evitar símbolos extraños
#simplemente aplicando tokenise fue suficiente para resolver el problema
from ipatok import tokenise
import os
import csv
#se retiró β, se reemplazaron tʃʼ,tʃʰ,tʃ por "č","čʰ","č'" 
# se añadio "/" dado que ese caracter acompaña al formato IPA en los archivos de la carpeta csv
IPAquechua=["æ","ɑ","a","ɪ","i","e","ɛ","u","U","o","ʊ","ɔ","h","j","k","k̚","ɣ","x","kʰ","kx","k'","l","ʎ","m","n","ɲ","p","ɸ","pʰ","pɸ","p'","q","ɢ","X","qʰ","q'","ɾ","s","ʃ","t","tʰ","ts","θ","t'","č","čʰ","č'","w","W"]
NoAlFinalPalabra=["h","ñ","č'","čʰ","kʰ","k'","pʰ","p'","qʰ","q'","t","tʰ","t'"] 
alofonos=["ɛ","e","o","ɔ"]
uvular=["q","q'","qʰ"]
vowels=["æ","ɑ","a","ɪ","i","e","ɛ","u","U","o","ʊ","ɔ"]
consonants=["h","j","k","k̚","ɣ","x","kʰ","kx","k'","l","ʎ","m","n","ɲ","p","β","ɸ","pʰ","pɸ","p'","q","ɢ","X","qʰ","q'","ɾ","s","ʃ","t","tʰ","ts","θ","t'","č","čʰ","č'","w"]
directory='/home/alonso/PalabrasTraducidasTranscritasIPA2/'
filename="IPA_ces-Latn_cs_words_alpha_part1.csv"
#for filename in os.listdir(directory):
with open(directory+filename,'r') as csvfile:
    reader = csv.reader(csvfile,delimiter='\t')
    for index,english,word,pron in reader:
        #verificar que la palabra examinada solamente contenga fonemas quechuas
        count=0
        if "'͡" in pron:
            #if pron[i] == "'" and pron[i+1] == "\u0361":
            print(pron,char,"sucede")
            #else: 
             #   print(pron,char,"not sucede")        



['a', 'b', 'a', 't', '͡', 'z', 'l', 'ɪ'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'y', '́'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'e', '̌'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'y', '́'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'e', '̌'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'y', '́'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['a', 't͡s', 't', '͡', 'z', 'r', 'a'] y sucede
['a', 'ɡ', 'r', 'e', 'ɟ', 'ɪ', 't', 'o', 'v', 'a', 't', ',', ' ', 'p', 'o', 'f', 'e', '̌', 'r', '̌', 'ɪ', 't', ',', ' ', 'z', 'b', 'l', 'n', 'o', 'm', 'o', 't', '͡', 'z', 'ɲ', 'ɪ', 't'] y sucede
['a', 'ɡ', 'r

['a', 'l', 'l', 'o', 't', '͡', 'z', 'l', 'a', 's', 'e'] y sucede
['a', 'l', 'l', 'o', 't', '͡', 'z', 'l', 'a', 's', 'ɪ', 't', 'e'] y sucede
['f', 's', '̌', 'e', 'm', 'o', 't', '͡', 'z', 'n', 'y', '́'] y sucede
['f', 's', '̌', 'e', 'm', 'o', 't', '͡', 'z', 'n', 'e', '̌'] y sucede
['f', 's', '̌', 'e', 'm', 'o', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['a', 'l', 'm', 'o', 't', '͡', 'z', 'r', 'e', 'b', 'e'] y sucede
['f', 's', 't', 'r', '̌', 'i', '́', 't', '͡', 'z', 'n', 'o', 's', 't'] y sucede
['a', 'm', 'ɪ', 't', '͡', 'z', 'l', 'a', 'e', 'a', 'n'] y sucede
['a', 'm', 'ɪ', 't', '͡', 'z', 'l', 'a', 's'] y sucede
['a', 'm', 'ɪ', 't', '͡', 'z', 'r', 'o', 'n'] y sucede
['ɲ', 'ɪ', 't', '͡', 'z', 'm', 'e', '́', 'n', 'e', '̌'] y sucede
['a', 'm', 'b', 'ɦ', 'ɪ', 'n', 'u', 't', '͡', 'z', 'l', 'e', 'u', 's'] y sucede
['a', 'n', 'a', 't͡s', 'ɪ', 't', '͡', 'z', 'l', 'u', 's'] y sucede
['a', 'n', 'a', 't', '͡', 'z', 'l', 'a', 's', 'ɪ', 's'] y sucede
['a', 'n', 'a', 't', '͡', 'z', 'l', 'e', 'c', 'ɪ

['e', 'p', 'a', 't', '͡', 'z', 'm', 'a', 'ɪ', 't͡s'] y sucede
['e', 'p', 'a', 't', '͡', 'z', 'm', 'e'] y sucede
['e', 'p', 'a', 't', '͡', 'z', 'r', 'ɪ', 't'] y sucede
['e', 'p', 'a', 't', '͡', 'z', 'r', 'ɪ', 'd', 'a', 't͡s', 'e', 'a', 'e'] y sucede
['e', 'p', 'a', 't', '͡', 'z', 'r', 'ɪ', 'd', 'a', 't͡s', 'e', 'o', 'u', 's'] y sucede
['e', 'p', 'a', 't', '͡', 'z', 'r', 'ɪ', 's'] y sucede
['e', 'p', 'ɪ', 't', '͡', 'z', 'n', 'e', 'm', 'ɪ', 'a'] y sucede
['e', 'p', 'ɪ', 't', '͡', 'z', 'r', 'a', 'ɲ', 'ɪ', 'u', 's'] y sucede
['e', 'p', 'ɪ', 't', '͡', 'z', 'r', 'a', 't', 'e', 's'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'ɪ'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'o', 't'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'o', 'l'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'o', 'l', 'l'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'o', 'w', 'e', 'e'] y sucede
['e', 's', 't', '͡', 'z', 'r', 'u', 'a', 'ɡ', 'e'] y sucede
['j', 'i', '́', 't', '͡', 'z', 'n', 'u'] y sucede
['j', 'i', '́', 't', '͡', 'z', 'n', 'u'

In [2]:
#CHANGE FILE NAME AND MOVE TO ANOTHER DIRECTORY
#este script añade el codigo ISO15924 a los nombres de los archivos de un directorio
#el input es un filename que tiene un código ISO639-1
import os
import pandas as pd
directory="/home/alonso/prueba2/"
new_directory="/home/alonso/PalabrasTraducidasGoogleAPI2/"
df = pd.read_csv('ISO639google.csv', sep='\t')
for filename in os.listdir(directory):
# Get the current working directory
#current_directory = os.getcwd()
    language = filename[:2]
    for index, row in df.iterrows():
        if language == row['639-1']:
            ISO15924script = row['Code']
# Get the path of the file to be renamed
            file_path = os.path.join(directory, filename)
# Get the new name of the file
            new_file_name = ISO15924script+"_"+filename
# Rename the file
            os.rename(file_path, new_directory+new_file_name)

In [ ]:
#rename files, put ISO639-1 code at beginning of a filename 
import os
from pathlib import Path

def rename_files(directory):
  for file in Path(directory).glob("*.csv"):
    two_letters_code = file.stem[-2:]
    new_file_name = two_letters_code + "_" + file.stem[:-2]+".csv"
    new_file = Path(directory, new_file_name)
    os.rename(file, new_file)

if __name__ == "__main__":
  directory = "/home/alonso/Music/prueba2"
  rename_files(directory)

In [55]:
#Find files into a directory and move them away
import os
import shutil
directory="/home/alonso/PalabrasTraducidasTranscritasIPA/"
new_directory="/home/alonso/PalabrasTraducidasGoogleAPI/"
noprocesados="/home/alonso/noprocesados"
df = pd.read_csv('ISO639google.csv', sep='\t')
for filename in os.listdir(new_directory):
# Get the current working directory
#current_directory = os.getcwd()
    name = "IPA_"+filename
    if name not in os.listdir(directory):
        shutil.copyfile(os.path.join(new_directory, filename), os.path.join(noprocesados, filename))

In [2]:
import csv
import epitran
from epitran.backoff import Backoff
from csv import writer
from csv import reader
# Open the input_file in read mode and output_file in write mode
backoff = Backoff(['hin-Deva', 'eng-Latn', 'cmn-Hans','tha-Thai'], cedict_file='/media/alonso/3361-6630/neologisms/cedict_ts.u8')
with open('/media/alonso/3361-6630/neologisms/Oxford English Wordlist Sample-500_normalized.txt', 'r') as filetoread, \
        open('pruebaanchocaracter.tsv', 'w', newline='') as filetofill:
    # Create a csv.reader object from the input file object
    # el input es un archivo compuesto de palabras, una por cada fila
    csv_reader = reader(filetoread)
    # Create a csv.writer object from the output file object
    # delimiter = \t le da formato tsv al archivo
    csv_writer = writer(filetofill, delimiter='\t', lineterminator='\n')
    # Read each row of the input csv file as list
    for row in csv_reader:
     IPA = backoff.trans_list(row[0])
    #este procedimiento convierte la lista IPA en una cadena separada por aquello que se coloque entre las comillas
    #si se quiere que la cadena tenga espacios en blanco intercalados, se coloca un espacio en blanco entre las comillas
    # además la palabra y su representación IPA se colocan contiguos en la misma fila
     row.append(''.join(IPA))
    # Add the updated row / list to the output filetofill
     csv_writer.writerow(row) 

In [ ]:
#traducir al inglés y castellano las palabras obtenidas de las carpetas csv y tsvbroad
import os
import csv
import pandas as pd
from pathlib import Path
import six
from google.cloud import translate_v2 as translate

translate_client = translate.Client()

filename="filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologismsQichwaCleanedbyHand.tsv"
with open(filename,'r') as csvfile:
    reader = csv.reader(csvfile,delimiter='\t')
    for row in reader:
        if len(row) == 7:
            ordinal, english, filename, language, word, IPA, qichwa = row
            spanish=translate_client.translate(english, source_language='en', target_language='es')["translatedText"]
            with open("filetsvTranslatedbyGoogleIPAbyEpitranTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", "a") as f:
                 f.write(f"{ordinal}\t{filename[13:15]}\t{filename}\t{word}\t{IPA}\t{qichwa}\t{english}\t{spanish}\n")          

In [40]:
import os
import csv
import six
import pycountry
import pandas as pd
from pathlib import Path
from google.cloud import translate_v2 as translate

translate_client = translate.Client()

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('ISO639GoogleComplete2023.csv')

#filename="/media/alonso/3361-6630/neologisms/TSVcompactoISO639.csv"
filename="file4tsvIPAQichwaWords.tsv"
with open(filename,'r') as csvfile:
    reader = csv.reader(csvfile,delimiter='\t')
    for row in reader:
        if len(row) == 6:
            ordinal, filename, word, IPA, tokenised, qichwa = row
            iso639_3_code = filename[:3]
            try:
                language = pycountry.languages.get(alpha_3=iso639_3_code)
                iso639_1_code = language.alpha_2
                #check if language is in the Google's set of languages:
                if iso639_1_code in df['ISO-639 code'].values:
                    #count+=1
                    #print(f"ISO 639-1 code for {iso639_3_code}: {iso639_1_code}, {count}")
                    english=translate_client.translate(word, source_language=iso639_1_code, target_language='en')["translatedText"]        
                    spanish=translate_client.translate(word, source_language=iso639_1_code, target_language='es')["translatedText"]
                    with open("file4tsvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", "a") as f:
                        f.write(f"{ordinal}\t{iso639_3_code}\t{filename}\t{word}\t{tokenised}\t{qichwa}\t{english}\t{spanish}\n")
                else:
                    with open("file4tsvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", "a") as f:
                        f.write(f"{ordinal}\t{iso639_3_code}\t{filename}\t{word}\t{tokenised}\t{qichwa}\n")
            except:
                if iso639_3_code=="cmn": 
                    iso639_1_code="zh"
                    english=translate_client.translate(word, source_language=iso639_1_code, target_language='en')["translatedText"]        
                    spanish=translate_client.translate(word, source_language=iso639_1_code, target_language='es')["translatedText"]
                    with open("file4tsvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", "a") as f:
                        f.write(f"{ordinal}\t{iso639_3_code}\t{filename}\t{word}\t{tokenised}\t{qichwa}\t{english}\t{spanish}\n")
                else: 
                    with open("file4tsvTokenisedbyIPAtokSelectedNeologismsQichwaEnglishSpanish.tsv", "a") as f:
                        f.write(f"{ordinal}\t{iso639_3_code}\t{filename}\t{word}\t{tokenised}\t{qichwa}\n")
                    continue
